# Insurance Claims Data Exploration

## Data Acquisition
First things first. Download the dataset using kagglehub.

In [ ]:

import kagglehub

# Download latest version
path = kagglehub.dataset_download("litvinenko630/insurance-claims")

print("Path to dataset files:", path)

Path to dataset files: /Users/unclebuiscuit/.cache/kagglehub/datasets/litvinenko630/insurance-claims/versions/1


Then use pandas to create a dataframe. You can view is usin Data Wrangler.

In [2]:
import pandas as pd

df = pd.read_csv(f"{path}/Insurance claims data.csv")
# df.head()
# df.columns.tolist()

## Analysis

Reading the coloumn types, you'll see that the information pertains to automotaive insurance (not general insurance). We see vehicle specs, customer info, and claims_status (binary: claim or no claim). Focus will be placed on the likelihood of claim rates rather than claim amounts.

Now, let's understand our target variable: claim_status

In [3]:
# Check claim_status distribution. 
# This tells us what % of policies resulted in claims (could be imbalanced as claimed in Kaggle description)
print("Claim Status Distribution:")
print(df['claim_status'].value_counts())
print(f"\nClaim rate: {df['claim_status'].mean():.2%}")

Claim Status Distribution:
claim_status
0    54844
1     3748
Name: count, dtype: int64

Claim rate: 6.40%


### Customer Age Groups & Claim Rates

**Hypothesis**: I belive that drivers on the younger end will have fewer claims that those on the older end.

Note: There is no information for drivers below the age of 34 in this dataset.

**Insights**: Older drivers (56+) have a 32% higher claim rate than younger drivers (35-40). This is the opposite of the typical pattern and might be related to vehicle age or driving conditions.
Claim Rate by Age Group:
  - 35-40: 5.7% (lowest risk)
  - 41-45: 6.6%
  - 46-50: 6.6%
  - 51-55: 6.7%
  - 56+: 7.5% (highest risk)

In [4]:
# Create age groups
df['age_group'] = pd.cut(df['customer_age'],
                        bins=[0, 25, 35, 45, 55, 100],
                        labels=['18-25', '26-35', '36-45', '46-55', '55+'])

# Calculate claim rate by age group
age_analysis = df.groupby('age_group')['claim_status'].agg(['sum', 'count', 'mean'])
age_analysis.columns = ['total_claims', 'total_policies', 'claim_rate']
print("\nClaim Rate by Age Group:")
print(age_analysis)


Claim Rate by Age Group:
           total_claims  total_policies  claim_rate
age_group                                          
18-25                 0               0         NaN
26-35               174            2949    0.059003
36-45              1951           31873    0.061212
46-55              1235           18625    0.066309
55+                 388            5145    0.075413


/var/folders/qq/2vt9q_g92bb7lrqg_5mc33bc0000gn/T/ipykernel_3157/3301595157.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_analysis = df.groupby('age_group')['claim_status'].agg(['sum', 'count', 'mean'])


Notice how the age group for customers 18-25 are zero or NaN? That is becuase they don't exists in our dataset. At this time, the max age is 75 and the min age is 35. This is proven below.

In [5]:
# Check the actual age range in the data
print(f"Minimum age: {df['customer_age'].min()}")
print(f"Maximum age: {df['customer_age'].max()}")
print("\n Age distribution:")
print(df['customer_age'].describe())

Minimum age: 35
Maximum age: 75

 Age distribution:
count    58592.000000
mean        44.823935
std          6.935604
min         35.000000
25%         39.000000
50%         44.000000
75%         49.000000
max         75.000000
Name: customer_age, dtype: float64


Our age froups will be adjusted to fix the range 35-75. People over the age of 55 seem to have the highest claim rate.

In [6]:
# Create age groups that match range in dataset
df['age_group'] = pd.cut(df['customer_age'],
                         bins=[34, 40, 45, 50, 55, 75],
                         labels=['35-40', '41-45', '46-50', '51-55', '56+'])

# Calculate claim rate by age group
age_analysis = df.groupby('age_group', observed=True)['claim_status'].agg(['sum', 'count', 'mean'])
age_analysis.columns = ['total_claims', 'total_policies', 'claim_raite']
print("\n Claim Rate by Age Group")
print(age_analysis)


 Claim Rate by Age Group
           total_claims  total_policies  claim_raite
age_group                                           
35-40              1130           19814     0.057030
41-45               995           15008     0.066298
46-50               800           12093     0.066154
51-55               435            6532     0.066595
56+                 388            5145     0.075413


### Region & Claim Rates

Note: Regions are some numeric code.

**Hypothesis**: Regions will have varying claim amounts. 

**Insight**: Region matters a lot with some regions have 3x higher claim rates than others.
- Highest risk region (35036): 10.7% - nearly double the average!
- Lowest risk region (65567): 3.9% - less than half the average
- Wide variation across regions (3.9% to 10.7%)

In [7]:
# Claim rate by region density.
region_analysis = df.groupby('region_density')['claim_status'].agg(['sum', 'count', 'mean'])
region_analysis.columns = ['total_claims', 'total_policies', 'claim_rate']
print('\n Claim Rate by Region Density')
print(region_analysis.sort_values('claim_rate', ascending=False))


 Claim Rate by Region Density
                total_claims  total_policies  claim_rate
region_density                                          
35036                     26             242    0.107438
16733                     17             207    0.082126
7788                     281            3660    0.076776
21622                     51             665    0.076692
3264                      29             379    0.076517
27742                     71             952    0.074580
4076                     433            6101    0.070972
27003                    520            7342    0.070825
8794                     954           13654    0.069870
13051                     55             890    0.061798
6108                      72            1212    0.059406
34738                    403            6979    0.057745
16206                     23             401    0.057357
5410                     195            3423    0.056968
34791                     87            1589    0.054751


### Safety Features & Claims

**Hypothesis**: Some safety feature (brake assist, parking sesnors) may reduce the number of claims as they may reduce risk of any accidents.

**Insight**: They surprising don't seem to reduce claims as I was expecting. Safer cars might be driven more aggressively or newer/expensive cars have more features AND more claims.

Airbags:
- 1 airbag: 6.0%
- 2 airbags: 6.4%
- 6 airbags: 6.5%

More airbags = slightly MORE claims (counterintuitive)

ESC (Electronic Stability Control):
- No ESC: 6.3%
- Yes ESC: 6.5%

ESC doesn't reduce claims

Brake Assist:
- No: 6.1%
- Yes: 6.6%

Brake assist = MORE claims

Parking Sensors:
- No: 5.4%
- Yes: 6.4%

Parking sensors = MORE claims

In [8]:
# Analyze impact of key safety features
safety_features = ['airbags', 'is_esc', 'is_brake_assist', 'is_parking_sensors']

print("\n Claim Rate by Safety Features: \n")
for feature in safety_features:
    if feature in df.columns:
        feature_analysis = df.groupby(feature)['claim_status'].agg(['sum','count','mean'])
        feature_analysis.columns = ['total_claims','total_policies','claim_rate']
        print(f"{feature}:")
        print(feature_analysis)
        print()


 Claim Rate by Safety Features: 

airbags:
         total_claims  total_policies  claim_rate
airbags                                          
1                  73            1209    0.060380
2                2573           40425    0.063649
6                1102           16958    0.064984

is_esc:
        total_claims  total_policies  claim_rate
is_esc                                          
No              2551           40191    0.063472
Yes             1197           18401    0.065051

is_brake_assist:
                 total_claims  total_policies  claim_rate
is_brake_assist                                          
No                       1612           26415    0.061026
Yes                      2136           32177    0.066383

is_parking_sensors:
                    total_claims  total_policies  claim_rate
is_parking_sensors                                          
No                           128            2373    0.053940
Yes                         3620           5621

### Vehicle Age & Claims
Is there a correlation between the age of a vehicle and there was a claim?

**Hypothesis**: Older vehicles are more worn than newer ones. The more worn something is, the more dimished it's effectiveness is. I believe there were will more cliams for older vehicles.

**Insight**: My hypothesis was incorrect. Reality is that newer vehicles will have higher claims. Brand new vehicles (0 years) will have 10.6% risk. Behicles between 0.2-2.8 years have a moderate risk ranging from 4.6-7%. Vehicles of 3+ years have the lowest risk, but take into consideration that it has a smaller sample size. 
- New car owners may drive more
- Higher reported values = more claims filed
- Less experience with the specific vehicle

Think about it, if you buy a new car, odds are you are in debt and have to make payments for it. While you are legally required to have insurance on it, it still beneficial to make claims on it to cover any cost from any accident. Ex: I total a $45,750 truck and I still owe $25,777. I need insurance to cover most of it.

In [9]:
# Vehicle age analysis
vehicle_age_analysis = df.groupby('vehicle_age')['claim_status'].agg(['sum','count','mean'])
vehicle_age_analysis.columns = ['total_claims','total_plicies','claim_rate']
print("\n Claim Rate by Vehicle Age")
print(vehicle_age_analysis.sort_values('vehicle_age'))


 Claim Rate by Vehicle Age
             total_claims  total_plicies  claim_rate
vehicle_age                                         
0.0                   559           5257    0.106334
0.2                   295           6362    0.046369
0.4                   313           5189    0.060320
0.6                   272           4415    0.061608
0.8                   248           3763    0.065905
1.0                   226           3342    0.067624
1.2                   206           3053    0.067475
1.4                   206           2888    0.071330
1.6                   174           2730    0.063736
1.8                   167           2643    0.063186
2.0                   215           3123    0.068844
2.2                   166           2442    0.067977
2.4                   153           2506    0.061053
2.6                   140           2315    0.060475
2.8                   124           2151    0.057648
3.0                    87           1964    0.044297
3.2               

### Customer Segementation
High-risk vs Low-risk profiles will now be taken into perspective. Risk will based off customer age and vehicle age.

**Hypothesis**: Midde-aged customers with new cars will be profiled with high-risk. Our data from earlier showed customers of age range 35-40 and newer vehicles within 3 years held higher claim rates.

**Insight**: High-risk customers (older drivers with brand new cars) have a 62% higher claim rate than low-risk customers
- Low Risk: 6.0% claim rate (30,781 policies)
- Medium Risk: 5.9% claim rate (20,746 policies)
- High Risk: 9.7% claim rate (7,065 policies)

In [10]:
# Create risk score based on findings
df['risk_score'] = 0

# Add risk points based on our findings
df.loc[df['customer_age'] >= 56, 'risk_score'] += 1  # Older customers
df.loc[df['vehicle_age'] == 0, 'risk_score'] += 2    # Brand new cars
df.loc[df['vehicle_age'] < 1, 'risk_score'] += 1     # Very new cars

# Segment customers
df['risk_segment'] = pd.cut(df['risk_score'],
                            bins=[-1, 0, 1, 4],
                            labels=['Low Risk', 'Medium Risk', 'High Risk'])

# Analyze segments
segment_analysis = df.groupby('risk_segment', observed=True)['claim_status'].agg(['sum', 'count', 'mean'])
segment_analysis.columns = ['total_claims', 'total_policies', 'claim_rate']
print("\nCustomer Risk Segmentation:")
print(segment_analysis)


Customer Risk Segmentation:
              total_claims  total_policies  claim_rate
risk_segment                                          
Low Risk              1847           30781    0.060005
Medium Risk           1216           20746    0.058614
High Risk              685            7065    0.096957


### Export data for Power BI

In [13]:
# Export dataframe with new features (age_group, risk_score, risk_segment)
export_path = "../insurance_claims_analyzed.csv"
df.to_csv(export_path, index=False)
print(f"Data exported successfully for Power BI to {export_path}")
print(f"File Loation: {export_path}")
print(f"total rows: {len(df)}")
print(f"total columns: {len(df.columns)}")

Data exported successfully for Power BI to ../insurance_claims_analyzed.csv
File Loation: ../insurance_claims_analyzed.csv
total rows: 58592
total columns: 44


In [19]:
 # Create a safety features summary for Power BI
safety_summary = pd.DataFrame([
    {'Feature': 'ESC: No', 'Claim_Rate': df[df['is_esc'] == 'No']['claim_status'].mean()},
    {'Feature': 'ESC: Yes', 'Claim_Rate': df[df['is_esc'] == 'Yes']['claim_status'].mean()},
    {'Feature': 'Brake Assist: No', 'Claim_Rate': df[df['is_brake_assist'] == 'No']['claim_status'].mean()},
    {'Feature': 'Brake Assist: Yes', 'Claim_Rate': df[df['is_brake_assist'] == 'Yes']['claim_status'].mean()},
    {'Feature': 'Parking Sensors: No', 'Claim_Rate': df[df['is_parking_sensors'] == 'No']['claim_status'].mean()},
    {'Feature': 'Parking Sensors: Yes', 'Claim_Rate': df[df['is_parking_sensors'] == 'Yes']['claim_status'].mean()},
])

# # Export this summary
# safety_summary.to_csv('../safety_features_analysis.csv', index=False)
# print(safety_summary)

# Re-export with explicit numeric type
safety_summary['Claim_Rate'] = safety_summary['Claim_Rate'].astype(float)
safety_summary.to_csv('../safety_features_analysis.csv', index=False)
print("Re-exported with numeric claim rates")
print(safety_summary.dtypes)  # Verify Claim_Rate is float64


Re-exported with numeric claim rates
Feature        object
Claim_Rate    float64
dtype: object
